# Transformer2.0 Chatbot


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
# try:
#   # The %tensorflow_version magic only works in colab.
#   %tensorflow_version 2.x
# except Exception as err:
#     print(str(err))
#     pass
!pip install --upgrade keras
from tensorflow import keras
import os
import sys
import json
import numpy as np
import pandas as pd
import tensorflow as tf
# tf.random.set_seed(1234)
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import text
from keras import metrics
from keras import backend

# tf version check
print(f'tf version : {tf.__version__}')

device = tf.test.gpu_device_name()
if device != '/device:GPU:0':
    print('GPU device not found')
print(f'GPU at : {device}')

     |████████████████████████████████| 378kB 4.8MB/s 
  Found existing installation: Keras 2.2.5
    Uninstalling Keras-2.2.5:
      Successfully uninstalled Keras-2.2.5


Using TensorFlow backend.


tf version : 1.15.0
GPU at : /device:GPU:0


In [0]:
# Tanhsiri 모듈, KhaiiiRestore모듈이 포함된 TransformerChatbot 깃 클론
# Tokenizer, Dataset, Transformer 모듈 포함
!git clone https://github.com/yoonjong12/TransformerChatbot.git
# from TransformerChatbot.Transformer.Tanhsiri import *

Cloning into 'TransformerChatbot'...
remote: Enumerating objects: 67, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 67 (delta 25), reused 49 (delta 13), pack-reused 0
Unpacking objects: 100% (67/67), done.


## 모듈
* 모듈 전체는 Tanhsiri py, Khaiii_restore 파일로 저장했음
* 그냥 깃헙이나 모듈로 import했으면 참고하고 넘어가시면 되겠습니다.
* 혹시 몰라서 노트북에는 남겨놓습니다.

In [0]:
from tensorflow import keras
import os
import json
import numpy as np
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import text

# Preprocess - 토크나이저와 전처리를 맡는 클래스! 순서대로 진행 추천
# 먼저 3개 인자 넣음 (Q와 A는 pandas series type일 때 작동확인)
# 1. buildTokenizer : 토크나이저 리턴(save 가능)
# 2. tokenize_and_filter : 토크나이징 된 input과 output 리턴

class Preprocess:
    def __init__(self):
        super().__init__()
    def buildTokenizer(self, questions, answers):
        corpus = questions + answers
        tk = Tokenizer(filters='!"#$%&()*+,-.s:;<=>?@[\\]^_`{|}~\t\n', lower=False) # / 만 살렸다, 소문자변환 안한다.
        tk.fit_on_texts(corpus)
        VOCAB_SIZE = len(tk.word_index) + 1
        self.START_TOKEN, self.END_TOKEN = [VOCAB_SIZE], [VOCAB_SIZE + 1]
        self.VOCAB_SIZE = VOCAB_SIZE + 2   
        return tk

    def saveTokenizer(self, directory, tokenizer):
        tk_json = tokenizer.to_json()
        with open(directory, 'w', encoding='UTF-8-sig') as f:
            f.write(json.dumps(vars(tokenizer), ensure_ascii = False))
    def loadTokenzier(self, directory):
        with open(directory, encoding='UTF-8-sig') as fh:
            data = json.load(fh)
        tk = Tokenizer()
        key = list(data.keys())
        for i in key:
            setattr(tk, i, data[i])
        VOCAB_SIZE = len(tk.word_index) + 1 
        self.START_TOKEN, self.END_TOKEN = [VOCAB_SIZE], [VOCAB_SIZE + 1]
        self.VOCAB_SIZE = VOCAB_SIZE + 2  
        return tk

    # 리스트 형태를 토큰 모델에 넣어주려고 text로 만듦
    def list2text(self, x):
        return [e for s in x for e in s]

    # Tokenize, filter and pad sentences
    def tokenize_and_filter(self, questions, answers, tokenizer, MAX_LENGTH=30):
        tokenized_inputs, tokenized_outputs = [], []

        for (sentence1, sentence2) in zip(questions, answers):
            # tokenize sentence
            sentence1 = [self.START_TOKEN] + tokenizer.texts_to_sequences(sentence1) + [self.END_TOKEN]
            sentence2 = [self.START_TOKEN] + tokenizer.texts_to_sequences(sentence2) + [self.END_TOKEN]
            # check tokenized sentence max length
            if len(sentence1) <= MAX_LENGTH and len(sentence2) <= MAX_LENGTH:
                tokenized_inputs.append(self.list2text(sentence1))
                tokenized_outputs.append(self.list2text(sentence2))

        # pad tokenized sentences
        tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(
                            tokenized_inputs, maxlen=MAX_LENGTH, padding='post')
        tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(
                            tokenized_outputs, maxlen=MAX_LENGTH, padding='post')
        return tokenized_inputs, tokenized_outputs
    
    def buildDataset(self, inputs, outputs, BATCH_SIZE, BUFFER_SIZE=20000):
        dataset = tf.data.Dataset.from_tensor_slices((
            {
                'inputs': inputs,
                'dec_inputs': outputs[:, :-1]
            },
            {
                'outputs': outputs[:, 1:]
            },
        ))
        dataset = dataset.cache()
        dataset = dataset.repeat()
        dataset = dataset.shuffle(BUFFER_SIZE)
        dataset = dataset.batch(BATCH_SIZE)
        dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
        return dataset

############################
#### MultiHeadAttention ####
############################
def scaled_dot_product_attention(query, key, value, mask):
    """Calculate the attention weights. """
    matmul_qk = tf.matmul(query, key, transpose_b=True)

    # scale matmul_qk
    depth = tf.cast(tf.shape(key)[-1], tf.float32)
    logits = matmul_qk / tf.math.sqrt(depth) # scaling 

    # add the mask to zero out padding tokens
    if mask is not None:
        logits += (mask * -1e9)

    # softmax is normalized on the last axis (seq_len_k)
    attention_weights = tf.nn.softmax(logits, axis=-1) 

    output = tf.matmul(attention_weights, value)

    return output

class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, name="multi_head_attention", **kwargs):
        super(MultiHeadAttention, self).__init__(**kwargs)
        self.num_heads = num_heads
        self.d_model = d_model
        self.depth = self.d_model // self.num_heads
        assert d_model % self.num_heads == 0
        self.query_dense = tf.keras.layers.Dense(units=d_model)
        self.key_dense = tf.keras.layers.Dense(units=d_model)
        self.value_dense = tf.keras.layers.Dense(units=d_model)
        self.dense = tf.keras.layers.Dense(units=d_model)

    def get_config(self):
        config = super(MultiHeadAttention, self).get_config()
        config.update({
                        "num_heads":self.num_heads, 
                        "d_model":self.d_model,
                       })
        return config

    def split_heads(self, inputs, batch_size):
        inputs = tf.reshape(inputs, shape=(batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(inputs, perm=[0, 2, 1, 3])

    def call(self, inputs):
        query, key, value, mask = inputs['query'], inputs['key'], inputs[ 
                                    'value'], inputs['mask']
        batch_size = tf.shape(query)[0]
        # linear layers
        query = self.query_dense(query)
        key = self.key_dense(key)
        value = self.value_dense(value)
        # split heads
        query = self.split_heads(query, batch_size)
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)
        # scaled dot-product attention
        scaled_attention = scaled_dot_product_attention(query, key, value, mask)
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])
        # concatenation of heads
        concat_attention = tf.reshape(scaled_attention,
                                        (batch_size, -1, self.d_model))
        outputs = self.dense(concat_attention)
        return outputs

#################
#### Masking ####
#################
def create_padding_mask(x):
    mask = tf.cast(tf.math.equal(x, 0), tf.float32)
    # (batch_size, 1, 1, sequence length)
    return mask[:, tf.newaxis, tf.newaxis, :]
def create_look_ahead_mask(x):
    seq_len = tf.shape(x)[1]
    look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
    padding_mask = create_padding_mask(x)
    return tf.maximum(look_ahead_mask, padding_mask)

######################
#### Pos Encoding ####
######################
class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, position, d_model, name='PositionalEncoding', **kwargs):
        super(PositionalEncoding, self).__init__(**kwargs)
        self.d_model = d_model
        self.position = position
        self.pos_encoding = self.positional_encoding(position, d_model)

    def get_angles(self, position, i, d_model):
        angles = 1 / tf.pow(tf.constant(10000,dtype='float32'), (2 * (i // 2)) / tf.cast(d_model, tf.float32))
        return position * angles

    def positional_encoding(self, position, d_model):
        angle_rads = self.get_angles(
                            position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
                            i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
                            d_model=d_model)
        # apply sin to even index in the array
        sines = tf.math.sin(angle_rads[:, 0::2])
        # apply cos to odd index in the array
        cosines = tf.math.cos(angle_rads[:, 1::2])

        pos_encoding = tf.concat([sines, cosines], axis=-1)
        pos_encoding = pos_encoding[tf.newaxis, ...]
        return tf.cast(pos_encoding, tf.float32)

    def get_config(self):
        config = super(PositionalEncoding, self).get_config().copy()
        config.update({
                        'd_model': self.d_model,
                        'position': self.position
                        })
        return config

    def call(self, inputs):
        return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

###########################
#### 인코더 내부 Layer ####
###########################
def encoder_layer(units, d_model, num_heads, dropout, name="encoder_layer"):
    inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
    padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

    attention = MultiHeadAttention(
        d_model, num_heads, name="attention")({
            'query': inputs,
            'key': inputs,
            'value': inputs,
            'mask': padding_mask
        })
    attention = tf.keras.layers.Dropout(rate=dropout)(attention)
    attention = tf.keras.layers.LayerNormalization(epsilon=1e-6)(inputs + attention)

    outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention)
    outputs = tf.keras.layers.Dense(units=d_model)(outputs)
    outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
    outputs = tf.keras.layers.LayerNormalization(epsilon=1e-6)(attention + outputs)

    return tf.keras.Model(
        inputs=[inputs, padding_mask], outputs=outputs, name=name)
  
##############################################
#### 임베딩 + Pos Encoding + 인코더 Layer ####
##############################################
def encoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name="encoder"):
    inputs = tf.keras.Input(shape=(None,), name="inputs")
    padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

    embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
    embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

    outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

    for i in range(num_layers):
        outputs = encoder_layer(
            units=units,
            d_model=d_model,
            num_heads=num_heads,
            dropout=dropout,
            name="encoder_layer_{}".format(i),
        )([outputs, padding_mask])

    return tf.keras.Model(
        inputs=[inputs, padding_mask], outputs=outputs, name=name)

###########################
#### 디코더 내부 Layer ####
###########################
def decoder_layer(units, d_model, num_heads, dropout, name="decoder_layer"):
    inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
    enc_outputs = tf.keras.Input(shape=(None, d_model), name="encoder_outputs")
    look_ahead_mask = tf.keras.Input(shape=(1, None, None), name="look_ahead_mask")
    padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

    attention1 = MultiHeadAttention(d_model, num_heads, name="attention_1")(inputs={
                    'query': inputs,
                    'key': inputs,
                    'value': inputs,
                    'mask': look_ahead_mask
                })
    attention1 = tf.keras.layers.LayerNormalization(
    epsilon=1e-6)(attention1 + inputs)

    attention2 = MultiHeadAttention(d_model, num_heads, name="attention_2")(inputs={
                    'query': attention1,
                    'key': enc_outputs,
                    'value': enc_outputs,
                    'mask': padding_mask
                })
    attention2 = tf.keras.layers.Dropout(rate=dropout)(attention2)
    attention2 = tf.keras.layers.LayerNormalization(
    epsilon=1e-6)(attention2 + attention1)

    outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention2)
    outputs = tf.keras.layers.Dense(units=d_model)(outputs)
    outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
    outputs = tf.keras.layers.LayerNormalization(
    epsilon=1e-6)(outputs + attention2)

    return tf.keras.Model(
                    inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
                    outputs=outputs,
                    name=name)

###################################################
#### 임베딩 + Pos Encoding + 디코더 내부 Layer ####
###################################################
def decoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name='decoder'):
    inputs = tf.keras.Input(shape=(None,), name='inputs')
    enc_outputs = tf.keras.Input(shape=(None, d_model), name='encoder_outputs')
    look_ahead_mask = tf.keras.Input(shape=(1, None, None), name='look_ahead_mask')
    padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

    embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
    embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

    outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

    for i in range(num_layers):
        outputs = decoder_layer(
                    units=units,
                    d_model=d_model,
                    num_heads=num_heads,
                    dropout=dropout,
                    name='decoder_layer_{}'.format(i),
                )(inputs=[outputs, enc_outputs, look_ahead_mask, padding_mask])

    return tf.keras.Model(
                        inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
                        outputs=outputs,
                        name=name)

#######################################
#### 인코더 + 디코더 = Transformer ####
#######################################
def transformer(vocab_size,
                num_layers,
                units,
                d_model,
                num_heads,
                dropout,
                name="transformer"):
    inputs = tf.keras.Input(shape=(None,), name="inputs")
    dec_inputs = tf.keras.Input(shape=(None,), name="dec_inputs")

    enc_padding_mask = tf.keras.layers.Lambda(
        create_padding_mask, output_shape=(1, 1, None),
        name='enc_padding_mask')(inputs)
    # mask the future tokens for decoder inputs at the 1st attention block
    look_ahead_mask = tf.keras.layers.Lambda(
        create_look_ahead_mask,
        output_shape=(1, None, None),
        name='look_ahead_mask')(dec_inputs)
    # mask the encoder outputs for the 2nd attention block
    dec_padding_mask = tf.keras.layers.Lambda(
        create_padding_mask, output_shape=(1, 1, None),
        name='dec_padding_mask')(inputs)

    enc_outputs = encoder(
                    vocab_size=vocab_size,
                    num_layers=num_layers,
                    units=units,
                    d_model=d_model,
                    num_heads=num_heads,
                    dropout=dropout,
                )(inputs=[inputs, enc_padding_mask])

    dec_outputs = decoder(
                    vocab_size=vocab_size,
                    num_layers=num_layers,
                    units=units,
                    d_model=d_model,
                    num_heads=num_heads,
                    dropout=dropout,
                )(inputs=[dec_inputs, enc_outputs, look_ahead_mask, dec_padding_mask])

    outputs = tf.keras.layers.Dense(units=vocab_size, name="outputs")(dec_outputs)

    return tf.keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)
    
#######################################
######### Loss & LearningRate #########
#######################################
def customLoss(MAX_LENGTH, name='customLoss'):
    def loss_function(y_true, y_pred, name='loss_function'):
        y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))

        loss = tf.keras.losses.SparseCategoricalCrossentropy(
            from_logits=True, reduction='none')(y_true, y_pred)

        mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
        loss = tf.multiply(loss, mask)
        return tf.reduce_mean(loss)
    return loss_function
    
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000, name='CustomSchedule', **kwargs):
        super(CustomSchedule, self).__init__(**kwargs)

        self.d_m = d_model
        self.d_model = tf.cast(d_model, tf.float32)
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps**-1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

    def get_config(self):
        config = {
                  'd_model': self.d_m,
                  'warmup_steps': self.warmup_steps,
                 }
        return config 

def custom_accuracy(MAX_LENGTH, name='custom_accuracy'):
    def accuracy(y_true, y_pred):
        y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
        return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)
    return accuracy

#### 예측 ####
def transform(sentence, analyzer, anl_type):
    if anl_type == 'Konlpy_morphs':
        return [[i] for i in  analyzer.morphs(sentence)]
    elif anl_type == 'Konlpy_pos':
        return [[i] for i in  analyzer.pos(sentence, join=True)]
    elif anl_type == 'Khaiii':
        sentence = [str(i).split('\t')[1] for i in  analyzer.analyze(sentence)]
        return [e for s in sentence for e in s.split(' + ')]
    else:
        print('use Konlpy or Khaiii')

def evaluate(model, tokenizer, sentence, analyzer, anl_type, MAX_LENGTH=30):
    VOCAB_SIZE = len(tokenizer.word_index) + 1
    START_TOKEN, END_TOKEN = [VOCAB_SIZE], [VOCAB_SIZE + 1]
    VOCAB_SIZE += 2

    sentence = [START_TOKEN] + tokenizer.texts_to_sequences(transform(sentence, analyzer, anl_type)) + [END_TOKEN]
    sentence = tf.expand_dims([e for s in sentence for e in s],axis=0)
    output = tf.expand_dims(START_TOKEN, 0)
    for i in range(MAX_LENGTH):
        predictions = model(inputs=[sentence, output], training=False)

    # select the last word from the seq_len dimension
        predictions = predictions[:, -1:, :]
        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

    # return the result if the predicted_id is equal to the end token
        if tf.equal(predicted_id, END_TOKEN[0]):
            break
        output = tf.concat([output, predicted_id], axis=-1)

    output = tf.squeeze(output, axis=0)
    return tokenizer.sequences_to_texts(
                        [[i] for i in output.numpy()])

def pos2text(token_list):
    pure_input = [i[:i.index('/')] for i in token_list]
    # 띄어쓰기 교정
    # Josa : 조사, PreEomi : 선어말어미, Eomi : 어미, Suffix : 접미사
    pos_list = ['Josa','PreEomi','Eomi','Suffix']
    for i in range(len(token_list)):
        p = token_list[i]
        if p[p.index('/')+1:] in pos_list:
            # '/토큰'으로 만들어서 나중에 앞 단어와 붙일 거임
            pure_input[i] = '/'+pure_input[i]
    pure_text = ' '.join(pure_input)
    pure_text = pure_text.replace(' /','')

    # Rule 추가
    pure_text = pure_text.replace('중 증','중증')
    pure_text = pure_text.replace(' 은','은')
    pure_text = pure_text.replace(' 로','로')
    pure_text = pure_text.replace(' 와','와')


    return pure_text

def token2text(token_list):
    return ' '.join(token_list)
def text2token(x, analyzer):
    return analyzer.pos(x, join=True)

In [0]:
from six import unichr
# from khaiii import KhaiiiApi

# Ref: https://github.com/bluedisk/hangul-toolkit/tree/master/hgtk
# 최종목표는 우리의 딥러닝모델이 Khaiii에 기반한 Sequence를 뱉을 때 자연스런 말로 바꿔주기입니다.
# compose : 'ㄴ','ㅏ','ㄴ' -->> '난'
# dempose : '난' -->> 'ㄴ','ㅏ','ㄴ'
# restore : '나','ㄴ' -->> '난'
# checkRestore : 'ㅂ니다' -->> 'ㅂ', '니다'
# token2sentence : Khaiii 토큰 -->> 자연스런 문장 
class Jamo:
    def __init__(self):
        self.CHO = (
            u'ㄱ', u'ㄲ', u'ㄴ', u'ㄷ', u'ㄸ', u'ㄹ', u'ㅁ', u'ㅂ', u'ㅃ', u'ㅅ',
            u'ㅆ', u'ㅇ', u'ㅈ', u'ㅉ', u'ㅊ', u'ㅋ', u'ㅌ', u'ㅍ', u'ㅎ'
        )

        self.JOONG = (
            u'ㅏ', u'ㅐ', u'ㅑ', u'ㅒ', u'ㅓ', u'ㅔ', u'ㅕ', u'ㅖ', u'ㅗ', u'ㅘ',
            u'ㅙ', u'ㅚ', u'ㅛ', u'ㅜ', u'ㅝ', u'ㅞ', u'ㅟ', u'ㅠ', u'ㅡ', u'ㅢ', u'ㅣ'
        )

        self.JONG = (
            u'', u'ㄱ', u'ㄲ', u'ㄳ', u'ㄴ', u'ㄵ', u'ㄶ', u'ㄷ', u'ㄹ', u'ㄺ',
            u'ㄻ', u'ㄼ', u'ㄽ', u'ㄾ', u'ㄿ', u'ㅀ', u'ㅁ', u'ㅂ', u'ㅄ', u'ㅅ',
            u'ㅆ', u'ㅇ', u'ㅈ', u'ㅊ', u'ㅋ', u'ㅌ', u'ㅍ', u'ㅎ'
        )

        self.JAMO = self.CHO + self.JOONG + self.JONG[1:]

        self.NUM_CHO = 19
        self.NUM_JOONG = 21
        self.NUM_JONG = 28

        self.FIRST_HANGUL_UNICODE = 0xAC00  # '가'
        self.LAST_HANGUL_UNICODE = 0xD7A3  # '힣'

    def compose(self, chosung, joongsung, jongsung=u''):
        if jongsung is None: jongsung = u''

        try:
            chosung_index = self.CHO.index(chosung)
            joongsung_index = self.JOONG.index(joongsung)
            jongsung_index = self.JONG.index(jongsung)
        except Exception:
            raise NotHangulException('No valid Hangul character index')

        return unichr(0xAC00 + chosung_index * self.NUM_JOONG *\
                    self.NUM_JONG + joongsung_index * self.NUM_JONG + jongsung_index)

    def decompose_index(self, code):
        jong = int(code % self.NUM_JONG)
        code /= self.NUM_JONG
        joong = int(code % self.NUM_JOONG)
        code /= self.NUM_JOONG
        cho = int(code)
        return cho, joong, jong

    def decompose(self, hangul_letter):
        code = ord(hangul_letter) - self.FIRST_HANGUL_UNICODE
        cho, joong, jong = self.decompose_index(code)

        if cho < 0:
            cho = 0
        try:
            return self.CHO[cho], self.JOONG[joong], self.JONG[jong]
        except:
            print("%d / %d  / %d"%(cho, joong, jong))
            print("%s / %s " %( self.JOONG[joong].encode("utf8"), self.JONG[jong].encode('utf8')))
            raise Exception()

    def restore(self, char1, char2):
        cho, joong,_ = self.decompose(char1)
        return self.compose(cho, joong, char2)

    def checkRestore(self, inp):
    # 만약 맨 앞글자가 자음 하나일 경우, 해당 자음과 나머지 글자 리턴
        if inp[0] in self.CHO:
            return inp[0], inp[1:]
        else:
            return 'Pass', 'Pass'

    # version 1
    # def token2sentence(self, text):
    #     api = KhaiiiApi()
    #     split_input = [str(i).split('\t') for i in api.analyze(text)]
    #     pure_input = [i[0] for i in split_input]
    #     pos_input = [i[1] for i in split_input]
    #     print(pos_input)
    #     new_pure = []

    #     # Restore '하','ㅂ니다' -->> '합','니다'
    #     for i in range(len(split_input)):
    #         inp_0, inp_1 = self.checkRestore(pure_input[i]) 
    #         if inp_0 != 'Pass':
    #             pure_input[i-1] = self.restore(pure_input[i-1], inp_0)
    #             pure_input[i] = inp_1

    # version 2
    def token2sentence(self, token_list):
        pos_input = token_list
        print(pos_input)
        pure_input = [i[:i.index('/')] for i in pos_input]
        print(pure_input)
        new_pure = []

        # Restore '하','ㅂ니다' -->> '합','니다'
        for i in range(len(token_list)):
            inp_0, inp_1 = self.checkRestore(pure_input[i]) 
            if inp_0 != 'Pass':
                try:
                    pure_input[i-1] = self.restore(pure_input[i-1], inp_0)
                except:
                    pass
                else:
                    pure_input[i] = inp_1

        # 띄어쓰기 교정
        # J : 조사, E : 어미, XS : 파생 접미사
        for i in range(len(token_list)):
            p = pos_input[i]
            if p[p.index('/')+1] == 'J' or p[p.index('/')+1] == 'E' or p[p.index('/')+1] == 'XS':
                new_pure[-1] = new_pure[-1] + pure_input[i]
            else:
                new_pure.append(pure_input[i])
        new_pure = [i for i in new_pure if len(i) > 0]
        pure_text = ' '.join(new_pure)

        # 우리 도메인에 맞게 '장애 인'을 붙여주자 
        pure_text = pure_text.replace('애 인','애인')

        return pure_text


## Tokenizer & Datasset

In [0]:
corpus = '/content/drive/My Drive/ToBigs/탄시리(Tanhsiri)/Phase1 : Data Collection & Preprocessing /Data/통합데이터/final_df.pickle'
df = pd.read_pickle(corpus)
# df.to_pickle('/content/drive/My Drive/ToBigs/chatbot_for_the_disabled/Phase1 : Data Collection & Preprocessing /Data/통합데이터/Konlpy_data.pickle')
df.head()

,Question,Answer,pre_Question,pre_Answer,Q_token,A_token,len_Q_token,len_A_token
0,중증장애아동수당을 받는 중증장애인이 만21세가 된 경우 장애인연금을 별도로 신청해야...,중증장애아동수당을 받는 중증장애인이 초중등교육법 제2조에 따른 학교에 재학중인 경우...,중증장애아동수당을 받는 중증장애인이 만21세가 된 경우 장애인연금을 별도로 신청해야...,중증장애아동수당을 받는 중증장애인이 초중등교육법 제2조에 따른 학교에 재학중인 경우...,"[중증, 장애, 아, 동, 수당, 을, 받는, 중, 증장, 애인, 이, 만, 21,...","[중증, 장애, 아, 동, 수당, 을, 받는, 중, 증장, 애인, 이, 초, 중등교...",25,65
1,장애인활동지원 교육기관과 제공기관에 대한 정보는 어디에서 확인할 수 있나요?,장애인활동지원 홈페이지(www.ableservice.or.kr)에서 지정된 활동 지...,장애인활동지원 교육기관과 제공기관에 대한 정보는 어디에서 확인할 수 있나요,장애인활동지원 홈페이지wwwableserviceorkr에서 지정된 활동 지원기관 및...,"[장애인, 활동, 지원, 교육, 기관, 과, 제, 공, 기관, 에, 대한, 정보, ...","[장애인, 활동, 지원, 홈페이지, wwwableserviceorkr, 에서, 지정...",19,21
2,기초연금 수급권을 포기하면 장애인연금 기초급여를 받을 수 있나요?,장애인연금법 제6조 제5항에 따라 기초연금 수급권자에게는 기초급여를 지급하지 않습니다.,기초연금 수급권을 포기하면 장애인연금 기초급여를 받을 수 있나요,장애인연금법 제6조 제5항에 따라 기초연금 수급권자에게는 기초급여를 지급하지 않습니다,"[기초, 연금, 수급, 권, 을, 포기, 하면, 장애인, 연금, 기초, 급여, 를,...","[장애인, 연금, 법, 제, 6조, 제, 5, 항, 에, 따라, 기초, 연금, 수급...",15,22
3,만19세 중증장애인이 2월에 학교를 졸업하고 장애인연금을 신청하면 2월부터 연금을 ...,"장애인연금법에 따라 2월 중에는 장애인연금을 신청할 수 없으며, 미리 신청하더라도 ...",만19세 중증장애인이 2월에 학교를 졸업하고 장애인연금을 신청하면 2월부터 연금을 ...,장애인연금법에 따라 2월 중에는 장애인연금을 신청할 수 없으며 미리 신청하더라도 3...,"[만, 19, 세, 중, 증장, 애인, 이, 2월, 에, 학교, 를, 졸업, 하고,...","[장애인, 연금, 법, 에, 따라, 2월, 중, 에는, 장애인, 연금, 을, 신청,...",25,24
4,장애인연금은 누가 받을 수 있나요?,"만18세 이상의 등록 장애인 중 장애인연금법 상 중증장애인(종전 1, 2급 및 3급...",장애인연금은 누가 받을 수 있나요,만18세 이상의 등록 장애인 중 장애인연금법 상 중증장애인종전 1 2급 및 3급 중...,"[장애인, 연금, 은, 누가, 받을, 수, 있나요]","[만, 18, 세, 이상, 의, 등록, 장애인, 중, 장애인, 연금, 법, 상, 중...",7,32


In [0]:
df = df[['Question','Answer']]
df.head()

,Question,Answer
0,중증장애아동수당을 받는 중증장애인이 만21세가 된 경우 장애인연금을 별도로 신청해야...,중증장애아동수당을 받는 중증장애인이 초중등교육법 제2조에 따른 학교에 재학중인 경우...
1,장애인활동지원 교육기관과 제공기관에 대한 정보는 어디에서 확인할 수 있나요?,장애인활동지원 홈페이지(www.ableservice.or.kr)에서 지정된 활동 지...
2,기초연금 수급권을 포기하면 장애인연금 기초급여를 받을 수 있나요?,장애인연금법 제6조 제5항에 따라 기초연금 수급권자에게는 기초급여를 지급하지 않습니다.
3,만19세 중증장애인이 2월에 학교를 졸업하고 장애인연금을 신청하면 2월부터 연금을 ...,"장애인연금법에 따라 2월 중에는 장애인연금을 신청할 수 없으며, 미리 신청하더라도 ..."
4,장애인연금은 누가 받을 수 있나요?,"만18세 이상의 등록 장애인 중 장애인연금법 상 중증장애인(종전 1, 2급 및 3급..."


In [0]:
# for i in range(1000):
#     print(df.iloc[i])
a = df['Question']
b = df['Answer']

for i, j in zip(a,b):
    print(i , j) 

중증장애아동수당을 받는 중증장애인이 만21세가 된 경우 장애인연금을 별도로 신청해야 하나요? 중증장애아동수당을 받는 중증장애인이 초중등교육법 제2조에 따른 학교에 재학중인 경우에는 만20세까지만 장애아동수당을 지급받을 수 있어 만21세 생일이 속하는 달에 별도로 장애인연금을 신청해야 신청한 월부터 장애인연금을 지급받을 수 있습니다.
장애인활동지원 교육기관과 제공기관에 대한 정보는 어디에서 확인할 수 있나요? 장애인활동지원 홈페이지(www.ableservice.or.kr)에서 지정된 활동 지원기관 및 활동 지원 교육기관을 확인할 수 있습니다.
기초연금 수급권을 포기하면 장애인연금 기초급여를 받을 수 있나요? 장애인연금법 제6조 제5항에 따라 기초연금 수급권자에게는 기초급여를 지급하지 않습니다.
만19세 중증장애인이 2월에 학교를 졸업하고 장애인연금을 신청하면 2월부터 연금을 받을 수 있나요? 장애인연금법에 따라 2월 중에는 장애인연금을 신청할 수 없으며, 미리 신청하더라도 3월부터 급여가 지급됩니다.
장애인연금은 누가 받을 수 있나요? 만18세 이상의 등록 장애인 중 장애인연금법 상 중증장애인(종전 1, 2급 및 3급 중복 장애인)에 해당하는 자가 신청할 수 있습니다.
장애인연금은 매년 인상되나요? 장애인연금은 매년 그 전년도의 기초급여액에 전전년도 대비 전년도의 전국 소비자 물가 변동률을 반영하여 인상하고 있습니다.
특수학교 재학중인 19세 중증장애인이 장애인연금을 신청할 수 있나요? 만18세~20세의 중증장애인이 초중등 교육법 제2조에 따른 학교에 재학 중인 경우에는 장애인연금 신청 대상에서 제외됩니다.
국가유공자로서 상이등급을 가지고 있는데 장애인연금 신청 가능한가요? 장애인복지법에 따라 등록한 중증장애인이면 신청 가능합니다.
중증장애인이 근로소득이 있어도 장애인연금을 받을 수 있나요? 장애인연금은 본인과 배우자의 소득과 재산을 조사해서 선정기준액 이하일 경우 지원이 가능합니다.
장애인등록 신청과 동시에 장애인연금을 신청할 수 있나요? 장애인복지법 제32조

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [0]:
# 사람마다 디렉토리 다름 주의
tk_dir = '/content/drive/My Drive/ToBigs/탄시리(Tanhsiri)/Phase2 : Modeling & Algorithm & Advanced Modeling/Transformer_준걸_윤종/tokenizer_data_Konlpy.json'
corpus = '/content/drive/My Drive/ToBigs/탄시리(Tanhsiri)/Phase1 : Data Collection & Preprocessing /Data/통합데이터/Konlpy_data.pickle'
df = pd.read_pickle(corpus)

# Khaiii 썼을 떄 쓰는 거
# def cut(x):
#     return [i[:i.index('/')+3] for i in x]
# questions = df['pre_Question'].apply(cut)
# answers = df['pre_Answer'].apply(cut)
# questions = df['Q_kh']
# answers = df['A_kh']

# Konlpy 썼을 떄 쓰는 거
questions = df['Q_noPos']
answers = df['A_noPos']

MAX_LENGTH = 30
preprocess = Preprocess()

# 옵션1) 새로운 토크나이저 만들기
# tokenizer = preprocess.buildTokenizer(questions, answers)
# questions_seq, answers_seq = preprocess.tokenize_and_filter(questions, answers, tokenizer, MAX_LENGTH)
# 옵션2) 토크나이저 저장
# preprocess.saveTokenizer(tk_dir, tokenizer)
# 옵션3) 기존 토크나이저 로드
tokenizer = preprocess.loadTokenzier(tk_dir)
questions_seq, answers_seq = preprocess.tokenize_and_filter(questions, answers, tokenizer, MAX_LENGTH)

print('Vocab size: {}'.format(preprocess.VOCAB_SIZE))
print('Number of samples: {}'.format(len(questions_seq)))

Vocab size: 3663
Number of samples: 1043


In [0]:
# dataset
BATCH_SIZE = 64
dataset = preprocess.buildDataset(questions_seq, answers_seq, BATCH_SIZE)
dataset

<PrefetchDataset shapes: ({inputs: (None, 30), dec_inputs: (None, 29)}, {outputs: (None, 29)}), types: ({inputs: tf.int32, dec_inputs: tf.int32}, {outputs: tf.int32})>

## Train

In [0]:
tf.keras.backend.clear_session()

# parameters
NUM_LAYERS = 2
D_MODEL = 512
NUM_HEADS = 8
UNITS = 512
DROPOUT = 0.2
VOCAB_SIZE = preprocess.VOCAB_SIZE

model = transformer(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS,
    units=UNITS,a
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT)

learning_rate = CustomSchedule(D_MODEL)

optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

model.compile(optimizer=optimizer, loss=customLoss(MAX_LENGTH), metrics=[custom_accuracy(MAX_LENGTH)])

# 옵션) 모델 plot
# tf.keras.utils.plot_model(model, to_file='transformer.png', show_shapes=True)

In [0]:
# 옵션) 재학습을 위한 모델 로드
# path = '/content/drive/My Drive/ToBigs/chatbot_for_the_disabled/Phase2 : Modeling & Algorithm & Advanced Modeling/Transformer_준걸_윤종/tf_model_Konlpy.h5'
# MAX_LENGTH = 30
# custom_objects = {'PositionalEncoding':PositionalEncoding,
#                   'MultiHeadAttention':MultiHeadAttention,
#                   'CustomSchedule':CustomSchedule,
#                   'loss_function':customLoss(MAX_LENGTH),
#                   'accuracy':custom_accuracy(MAX_LENGTH), 
#                   'create_padding_mask':create_padding_mask,
#                   'backend':backend, 
#                   'tf':tf}
# model = keras.models.load_model(path, custom_objects=custom_objects)

In [0]:
steps = len(questions_seq) // BATCH_SIZE
EPOCHS = 150
print(f'MAX_LENGTH : {MAX_LENGTH}, EPOCHS : {EPOCHS}')
model.fit(dataset, epochs=EPOCHS, steps_per_epoch=steps)

# 옵션) verbose = 0 모드일 떄 최종 train acc 찍어보는 거
# test_scores = model.evaluate(dataset, verbose=2)
# print('Test loss:', test_scores[0])
# print('Test accuracy:', test_scores[1])

MAX_LENGTH : 30, EPOCHS : 150
Train for 16 steps
Epoch 1/150
16/16 [==============================] - 10s 607ms/step - loss: 4.4761 - accuracy: 6.7349e-05
Epoch 2/150
16/16 [==============================] - 1s 80ms/step - loss: 4.5235 - accuracy: 0.0011
Epoch 3/150
16/16 [==============================] - 1s 78ms/step - loss: 4.2649 - accuracy: 0.0263
Epoch 4/150
16/16 [==============================] - 1s 79ms/step - loss: 4.2054 - accuracy: 0.0345
Epoch 5/150
16/16 [==============================] - 1s 79ms/step - loss: 4.1190 - accuracy: 0.0345
Epoch 6/150
16/16 [==============================] - 1s 79ms/step - loss: 4.0365 - accuracy: 0.0345
Epoch 7/150
16/16 [==============================] - 1s 78ms/step - loss: 3.9097 - accuracy: 0.0361
Epoch 8/150
16/16 [==============================] - 1s 79ms/step - loss: 3.8171 - accuracy: 0.0387
Epoch 9/150
16/16 [==============================] - 1s 79ms/step - loss: 3.7582 - accuracy: 0.0422
Epoch 10/150
16/16 [=========================

In [0]:
# 모델 저장
path = '/content/drive/My Drive/ToBigs/탄시리(Tanhsiri)/Phase2 : Modeling & Algorithm & Advanced Modeling/Transformer_준걸_윤종/tf_model_noPos3.h5'
model.save(path)
del model

## Evaluate
* 위에서 라이브러리 import하고 넘어올 것
* 모델, 토크나이저 로드하고 Khaiii api, Khaiii restore 라이브러리 불러온다
* 예측하면 끝

In [0]:
# Install Khaiii
!git clone https://github.com/kakao/khaiii.git
!pip install cmake
!mkdir build
!cd build && cmake /content/khaiii
!cd /content/build/ && make all
!cd /content/build/ && make resource
!cd /content/build && make install
!cd /content/build && make package_python
!pip install /content/build/package_python 

Cloning into 'khaiii'...
remote: Enumerating objects: 877, done.
remote: Total 877 (delta 0), reused 0 (delta 0), pack-reused 877
Receiving objects: 100% (877/877), 30.21 MiB | 33.01 MiB/s, done.
Resolving deltas: 100% (358/358), done.
-- [hunter] Initializing Hunter workspace (70287b1ffa810ee4e952052a9adff9b4856d0d54)
-- [hunter]   https://github.com/ruslo/hunter/archive/v0.23.34.tar.gz
-- [hunter]   -> /root/.hunter/_Base/Download/Hunter/0.23.34/70287b1
-- The C compiler identification is GNU 7.4.0
-- The CXX compiler identification is GNU 7.4.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX comp

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-1226ec1ead70>", line 8, in <module>
    get_ipython().system('cd /content/build && make package_python')
  File "/usr/local/lib/python3.6/dist-packages/google/colab/_shell.py", line 96, in system
    output = _system_commands._system_compat(self, *args, **kwargs)  # pylint:disable=protected-access
  File "/usr/local/lib/python3.6/dist-packages/google/colab/_system_commands.py", line 438, in _system_compat
    shell.var_expand(cmd, depth=2), clear_streamed_output=False)
  File "/usr/local/lib/python3.6/dist-packages/google/colab/_system_commands.py", line 181, in _run_command
    delay_millis=500) as update_stdin_widget:
  File "/usr/lib/python3.6/contextlib.py", line 81, in __enter__
    return next(self.gen)
  File "/usr/local/lib/python3.6/dist-packages/google/colab/_s

KeyboardInterrupt: ignored

In [0]:
# Install Konlpy
!apt-get update
!apt-get install g++ openjdk-8-jdk 
!pip3 install konlpy

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:10 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:13 https://developer.download.nvidia.com/co

In [0]:
# 모델 로드
tf.keras.backend.clear_session()

path_1 = '/content/drive/My Drive/ToBigs/탄시리(Tanhsiri)/Phase2 : Modeling & Algorithm & Advanced Modeling/Transformer_준걸_윤종/tf_model_noPos3.h5'
# path_2 = '/content/drive/My Drive/ToBigs/탄시리(Tanhsiri)/Phase2 : Modeling & Algorithm & Advanced Modeling/Transformer_준걸_윤종/tf_model_noPos2.h5'

MAX_LENGTH = 30
custom_objects = {'PositionalEncoding':PositionalEncoding,
                  'MultiHeadAttention':MultiHeadAttention,
                  'CustomSchedule':CustomSchedule,
                  'loss_function':customLoss(MAX_LENGTH),
                  'accuracy':custom_accuracy(MAX_LENGTH), 
                  'create_padding_mask':create_padding_mask,
                  'backend':backend, 
                  'tf':tf}
model_1 = keras.models.load_model(path_1, custom_objects=custom_objects)
# model_2 = keras.models.load_model(path_2, custom_objects=custom_objects)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [0]:
# 토크나이저 로드
tk_dir_1 = '/content/drive/My Drive/ToBigs/탄시리(Tanhsiri)/Phase2 : Modeling & Algorithm & Advanced Modeling/Transformer_준걸_윤종/tokenizer_data_Konlpy_noPos.json'
# tk_dir_2 = '/content/drive/My Drive/ToBigs/탄시리(Tanhsiri)/Phase2 : Modeling & Algorithm & Advanced Modeling/Transformer_준걸_윤종/tokenizer_data_Konlpy_noPos.json'

preprocess1 = Preprocess()
# preprocess2 = Preprocess()

tokenizer_1 = preprocess1.loadTokenzier(tk_dir_1)
# tokenizer_2 = preprocess2.loadTokenzier(tk_dir_2)

# 토크나이저 로드하면 모든 key,value가 string으로 들어감 나중에 토큰을 텍스트로 
# 복원할 때 정상적으로 구동하기 위해서 index_word는 key를 int로 바꿔줌
tokenizer_1.index_word = {int(k):v for k,v in tokenizer_1.index_word.items()}
# tokenizer_2.index_word = {int(k):v for k,v in tokenizer_2.index_word.items()}

In [0]:
# 예측
# Khaiii
# from khaiii import KhaiiiApi
# from TransformerChatbot.Khaiii.KhaiiiRestore import Jamo
# api = KhaiiiApi()
# jamo = Jamo()
# Konlpy
from konlpy.tag import Kkma
from konlpy.tag import Okt
t = Okt()

while True:
    sentence = input()
    if sentence == 'quit':
        break
    else:
        prediction_1 = evaluate(model_1, tokenizer_1, sentence, t, anl_type='Konlpy_morphs', MAX_LENGTH=30)
        # prediction_2 = evaluate(model_2, tokenizer_2, sentence, t, anl_type='Konlpy_morphs', MAX_LENGTH=30)
        new_text_1 = pos2text(text2token(token2text(prediction_1), t))
        # new_text_2 = pos2text(text2token(token2text(prediction_2), t))

        print(f'Q : {sentence}')
        print(f'Konlpy 1 : {new_text_1}')
        # print(f'Konlpy 2 : {new_text_2}')

장애인연금은 본인만 신청할 수 있나요


OperatorNotAllowedInGraphError: ignored

In [0]:
corpus = '/content/drive/My Drive/ToBigs/chatbot_for_the_disabled/Phase1 : Data Collection & Preprocessing /Data/통합데이터/data.csv'
df = pd.read_csv(corpus)
for i in df['Q']:
    print(i)

기초 연금 수급권자는 장애인 연금 기초 급여를 받을 수 있나요
국가 유공자로서 상이등급을 가지고 있는데 장애인 연금 신청 가능한가요
중증 장애인이 근로 소득이 있어도 장애인 연금을 받을 수 있나요
65세 이상이 되면 장애인 연금 기초 급여를 지급하지 않는 이유는 무엇인가요
장애인연금은 본인만 신청할 수 있나요
장애인연금을 신청하면 장애 재심사를 반드시 받아야 하나요
장애인연금을 신청했는데 결정되기까지 얼마나 걸리나요
장애 수당을 신청하는 경우 장애 정도 재심사를 받아야 하나요
장애인 전용 주차 구역 주차 표지는 어디서 발급받나요
장애인 차량이 여러 대일 경우 주차 표지를 각각 발급 받을 수 있나요
장애인 활동지원 서비스 이용하려면 국민행복카드를 새로 발급 받아야 하나요
장애인이면 장애인 전용 주차구역에 주차가능한 표지를 발급 받을 수 있나요
오토바이가 장애인 전용 주차구역에 주차되어 있는 경우 주차방해행위로 볼 수 있나요
장애인연금은 어떻게 구분되나요
구청에서 장애인에게 제공하는 대중교통이 있나요
찾아가는 방문 재활사업에 대해 알려주세요
장애인연금 지급 조건에 대해서 알고 싶습니다
장애인 주차구역 불법 주차 신고는 어떻게 하나요
장애 등급제가 폐지된 후 장애인 주차표지 발급 기준이 궁금합니다
장애인 등록을 하려는데 어디에 신고해야 하나요
장애인보장구를 구입하려고 하는데 지원받을 수 있나요
장애인자동차표지 발급 신청 시 필요한 서류는 무엇입니까
장애등급제 폐지가 되면 장애인등록 절차 및 구비서류가 달라지나요
특수교육지원센터의 주요 역할이 무엇인가요
장애인 전용 주차구역에는 어떤 차량이 주차 가능한가요
장애인 주차 구역 위반 단속 시간은 어떻게 되나요
특수 교육 대상자 선정 후 의무 교육 기간동안 계속 지원을 받아야만 하나요
특수 교육 대상자로 선정되면 의무 교육 기간동안 계속 지원을 받아야만 하나요
장애등급이 있는 사람이 신호 위반에 단속되어 과태료 용지를 받았는데 감경이나 면제되는 방법이 있나요
 장애인증을 분실하였는데 장애인으로 과태료 감경을 신청할 수 있나

     |████████████████████████████████| 81kB 4.2MB/s 
  Created wheel for fasttext: filename=fasttext-0.8.3-cp36-cp36m-linux_x86_64.whl size=1092679 sha256=e490c78b48cde65d79aba2e9652aaf625bbfcd8dcbb21ccd9fc900922dfc4afc
  Stored in directory: /root/.cache/pip/wheels/73/8e/5d/ecb50b90adaab5868ae1d8df180f31e55e85c2f055aaf2fb35
Successfully built fasttext


In [0]:

def loadvector(File):
    f = open(File,'r')
    model = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    return model

def pred_correction_rnn(sent,model,dic,maxlen,wdim):
    conv = np.zeros((1,maxlen,wdim,1))
    rnn  = np.zeros((1,maxlen,wdim))
    charcount = -1
    for j in range(len(sent)):
      if j<maxlen and sent[j]!=' ':
        charcount=charcount+1
        conv[0][charcount,:,0]=dic[sent[j]]
        rnn[0][charcount,:]=dic[sent[j]]
    z = model.predict([conv,rnn])[0]
    sent_raw = ''
    count_char=-1
    lastpoint=-1
    lastchar=-1
    for j in range(len(sent)):
      if sent[j]!=' ':
        count_char=count_char+1
        sent_raw = sent_raw+sent[j]
        if z[count_char]>threshold_kor:
          sent_raw = sent_raw+' '
          if j<overlap:
            lastpoint=len(sent_raw)
            lastchar=j
    return sent_raw, lastpoint, lastchar

def c(s):
    if len(s)<overlap:
      temp,lp,lc = pred_correction_rnn(s,model_kor,dic_kor,100,100)
      z = temp+"\n"
    else:
      z=''
      start=0
      while start<len(s):
        if start+overlap<len(s):
          temp,lp,lc =pred_correction_rnn(s[start:start+2*overlap],model_kor,dic_kor,100,100)
          temp=temp[:lp]
        else:
          temp,lp,lc =pred_correction_rnn(s[start:],model_kor,dic_kor,100,100)
          lc = overlap
        z = z+temp
        start=start+lc+1
      z = z+"\n"
    return z

config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.01
set_session(tf.Session(config=config))

adam_half = optimizers.Adam(lr=0.0005)

dic_kor = fasttext.load_model('/content/drive/My Drive/model_kor.bin')

idchar = {}
for i in range(len(string.ascii_lowercase)):
  idchar.update({string.ascii_lowercase[i]:i})

for i in range(10):
  idchar.update({i:i+26})

idchar.update({'#':36})

big  = re.compile(r"[A-Z]")
small= re.compile(r"[a-z]")
num  = re.compile(r"[0-9]")

model_kor = load_model('/content/drive/My Drive/model_kor.hdf5')

## Functions_KOR

threshold_kor=0.5
overlap=30


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:384: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [0]:
kor_spacing('이제집에가고싶어요')

'이제 집에 가고 싶어요 \n'

In [0]:

print(keras.__version__)
print(tf.__version__)

2.2.4-tf
1.15.0
